In [2]:
!pip install catboost
!pip install --user --upgrade catboost

  Stored in directory: C:\Users\lenovo\AppData\Local\pip\Cache\wheels\d7\a9\33\acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying
Requirement already up-to-date: catboost in c:\users\lenovo\anaconda3\lib\site-packages (0.20)


In [3]:
!pip install shap
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [4]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=4)

In [5]:
import catboost
print(catboost.__version__)

0.20


In [6]:
from catboost.datasets import amazon

In [7]:
train_df,test_df=amazon()

In [8]:
train_df.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [9]:
test_df.head()

,id,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,2,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,3,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,4,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,5,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [10]:
import seaborn as sns

In [ ]:
sns.pairplot(train_df)

In [ ]:
sns.pairplot(test_df)

In [12]:
y=train_df.ACTION

In [13]:
x=train_df.drop('ACTION',axis=1)

In [45]:
cat_features=list(range(0,x.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [15]:
print('labels:{}'.format(set(y)))
print('zero count={},one count={}'.format(len(y)-sum(y),sum(y)))

labels:{0, 1}
zero count=1897,one count=30872


In [18]:
from catboost import CatBoostClassifier

In [31]:
model=CatBoostClassifier(iterations=100)

In [32]:
model.fit(x,y,cat_features=cat_features,verbose=10)

Learning rate set to 0.349945
0:	learn: 0.4668712	total: 94.6ms	remaining: 9.37s
10:	learn: 0.1779134	total: 1.15s	remaining: 9.27s
20:	learn: 0.1689247	total: 1.82s	remaining: 6.84s
30:	learn: 0.1660732	total: 2.52s	remaining: 5.62s
40:	learn: 0.1647466	total: 3.17s	remaining: 4.55s
50:	learn: 0.1630501	total: 3.77s	remaining: 3.63s
60:	learn: 0.1619123	total: 4.42s	remaining: 2.83s
70:	learn: 0.1606168	total: 5.04s	remaining: 2.06s
80:	learn: 0.1598533	total: 5.67s	remaining: 1.33s
90:	learn: 0.1592465	total: 6.3s	remaining: 623ms
99:	learn: 0.1587221	total: 6.87s	remaining: 0us


In [21]:
model.predict_proba(x)

array([[0.0074, 0.9926],
       [0.0094, 0.9906],
       [0.0431, 0.9569],
       ...,
       [0.0106, 0.9894],
       [0.1099, 0.8901],
       [0.0229, 0.9771]])

In [23]:
model.feature_importances_

array([20.6248, 25.549 ,  2.8729,  6.3637, 11.7977,  7.2495, 15.6688,
        5.2209,  4.6527])

In [24]:
model.classes_

['0', '1']

In [26]:
model.get_cat_feature_indices;

there are several ways of passing dataset to training using x,y or using pool class.pool class is the class of storing the dataset.

you can use pool class other than x,y. for example it has sample of weights or groups or if the dataset is large and it takes long time to read it into python.

In [34]:
from catboost import Pool

In [35]:
pool=Pool(data=x,label=y,cat_features=cat_features)

selecting the objective unction
possible options for binary classification:
logloss for binary target.
cross entropy for probabilities in target

In [37]:
model1=CatBoostClassifier(iterations=5,learning_rate=0.1,loss_function='CrossEntropy')

In [43]:
model1.fit(x_train,eval_set=x_test,verbose=1)

CatBoostError: y may be None only when X is an instance of catboost.Pool or string

In [40]:
from sklearn.model_selection import train_test_split

In [42]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [46]:
train_pool=Pool(data=x_train,label=y_train,cat_features=cat_features)

In [47]:
test_pool=Pool(data=x_test,label=y_test,cat_features=cat_features)

In [49]:
model1=CatBoostClassifier(iterations=5,learning_rate=0.1,loss_function='CrossEntropy')
model1.fit(train_pool,eval_set=test_pool,verbose=1)

0:	learn: 0.6186106	test: 0.6189138	best: 0.6189138 (0)	total: 128ms	remaining: 513ms
1:	learn: 0.5576742	test: 0.5581390	best: 0.5581390 (1)	total: 192ms	remaining: 288ms
2:	learn: 0.5071928	test: 0.5078888	best: 0.5078888 (2)	total: 305ms	remaining: 204ms
3:	learn: 0.4647982	test: 0.4656169	best: 0.4656169 (3)	total: 382ms	remaining: 95.5ms
4:	learn: 0.4289798	test: 0.4301941	best: 0.4301941 (4)	total: 479ms	remaining: 0us

bestTest = 0.4301940613
bestIteration = 4

